In [1]:
import pandas as pd

In [42]:
import urllib, urllib.request


In [43]:
from bs4 import BeautifulSoup

In [4]:
import re
def strip(s):
    s=' '.join(s.split())
    return re.sub(r'[^A-Za-z0-9 ]+', '', s)

In [5]:
import fitz
import io
import requests

DHQ scraping

In [45]:
url="http://www.digitalhumanities.org/dhq/index/title.html"

In [46]:
soup = BeautifulSoup(requests.get(url).text)

C:\Users\lucia\miniconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [57]:
count=0
links=[]
for para in soup.find_all("a"):
    count+=1
    if count<92:
        continue
    link=para['href']
    if link[0]=="/":
        links.append("http://www.digitalhumanities.org/"+link)
    

In [68]:
soup = BeautifulSoup(requests.get("http://www.digitalhumanities.org/dhq/vol/16/4/000643/000643.html").text)

In [117]:
out=[]
for l in links:
    text=""
    soup = BeautifulSoup(requests.get(l).text)
    try:
        date=soup.find_all("a")[66].get_text()
    except:
        continue
    title=strip(soup.find_all("h1")[0].get_text())

    authors=[]
    for para in soup.find_all("a"):
        try:
            if "../bios.html" in para["href"]:
                authors.append(strip(para.text))
        except:
            continue
    for para in soup.find_all("div",  {"class": "ptext"}):
        text+=para.get_text()
        text+=" "
    try:
        abstract=strip(soup.find_all("div", {"id": "abstract"})[0].get_text()[9:])
    except:
        abstract=""
    out.append([date,title, authors,abstract,strip(text), l])

In [119]:
df=pd.DataFrame(out)
df.columns=["date", "title","author", "abstract", "text", "url"]
df.to_csv("dhq.csv")

In [6]:
def annotate(i, src):
    date=i["published"]
    title=i["title"]
    authors=i["authors"]
    abstract=strip(i['summary'])
    url = i["links"][-1]["href"]
    tag=i["tags"]
    request = requests.get(url)
    filestream = io.BytesIO(request.content)
    pdf = fitz.open(stream=filestream, filetype="pdf")
    text = ""
    for page in pdf:
        text += page.get_text()
    return [date,title,tag, authors,abstract,strip(text),i["id"], src]
    

In [16]:
def parse_file(path):
    text=""
    doc = fitz.open(path, filetype="pdf")
    for page in doc:
        text += page.get_text()
    return text

In [272]:
def parse(url):
    text = ""
    try:
        request = requests.get(url)
        filestream = io.BytesIO(request.content)
        pdf = fitz.open(stream=filestream, filetype="pdf")
        for page in pdf:
            text += page.get_text()
    except:
        soup = BeautifulSoup(requests.get(url).text)
        [s.extract() for s in soup(['header',"footer",'style', 'script', '[document]', 'head', 'title'])]
        visible_text = soup.getText()
        for para in soup.find_all("p"):
            text+=para.get_text()
            text+=" "
    return text

In [9]:
df=pd.read_csv("open_source.csv")

In [231]:
#download based on keyword matches
urls=[]
urls.append(('https://doaj.org/api/search/articles/%22google%20books%22?pageSize=100', "goog"))
urls.append(('https://doaj.org/api/search/articles/%22google%20books%20ngram%20%22?pageSize=100', "goog"))
urls.append(('https://doaj.org/api/search/articles/%22google%20books%20n-gram%20%22?pageSize=100', "goog"))
urls.append(('https://doaj.org/api/search/articles/%22google%20ngram%20%22?pageSize=100', "goog"))
urls.append(('https://doaj.org/api/search/articles/%22google%20n-gram%20%22?pageSize=100', "goog"))

urls.append(('https://doaj.org/api/search/articles/%22hathitrust%22?pageSize=100', "hathi"))
urls.append(('https://doaj.org/api/search/articles/%22hathi%20trust%22?pageSize=100', "hathi"))

urls.append(('https://doaj.org/api/search/articles/%22internet%20archive%22?pageSize=100', "ia"))

urls.append(('https://doaj.org/api/search/articles/%22project%20gutenberg%22?pageSize=100', "guten"))


In [288]:
#download all Journal of Cultural Analytics
urls=[]
urls.append(("https://doaj.org/api/search/articles/bibjson.journal.title.exact%3A%22Journal%20of%20Cultural%20Analytics%22?pageSize=100", "jca"))


In [289]:
ret=[]
while urls:
    dat=urls.pop(0)
    url=dat[0]
    mat=dat[1]
    response = requests.get(url)
    if "next" in response.json():
        urls.append((response.json()["next"],mat))
    jr = response.json()["results"]
    for i in jr:
        date=i["created_date"]
        title=i["bibjson"]["title"]
        tag=i["bibjson"]["subject"]
        authors=i["bibjson"]["author"]
        try:
            abstract=i["bibjson"]["abstract"]
        except:
            abstract=""
        journal=i["bibjson"]["journal"]["title"]
        url=i["bibjson"]["link"][0]["url"]
        try:
            text=parse(url)
        except:
            text=""
        ret.append([date,title,tag, authors,abstract,strip(text),url, mat, journal])
    
    
    

In [290]:
df=pd.DataFrame(ret)
df.columns=["date", "title","tag","author", "abstract", "text", "url", "match", "journal"]

In [292]:
df=df.drop_duplicates(subset=["title"])

In [246]:
missing=df[df["text"]==""]

In [36]:
df.to_csv("open_source.csv")

In [5]:
import feedparser



In [62]:
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22internet+archive%22&max_results=10000')


In [87]:
def annotate(i, src):
    date=i["published"]
    title=i["title"]
    authors=i["authors"]
    abstract=strip(i['summary'])
    url = i["links"][-1]["href"]
    tag=i["tags"]
    request = requests.get(url)
    filestream = io.BytesIO(request.content)
    pdf = fitz.open(stream=filestream, filetype="pdf")
    text = ""
    for page in pdf:
        text += page.get_text()
    return [date,title,tag, authors,abstract,strip(text),i["id"], src]
    

In [89]:
ents=[]
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:hathitrust&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "hathi"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22internet+archive%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "ia"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22hathi+trust%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "hathi"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22google+books%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "goog"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22google+ngram%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "goog"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22google+books+ngram%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "goog"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22project+gutenberg%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "guten"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22google+n-gram%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "goog"))
feed = feedparser.parse('http://export.arxiv.org/api/query?search_query=all:%22google+books+n-gram%22&max_results=10000')
for i in feed.entries:
    ents.append(annotate(i, "goog"))

In [91]:
df=pd.DataFrame(ents)
df.columns=["date", "title","tag","author", "abstract", "text", "url", "match"]

In [133]:
df.to_csv("arxiv.csv")